In [1]:
# SageMath version at least 10.4
%display latex
version()

'SageMath version 10.4, Release Date: 2024-07-19'

In [2]:
load("cayley-dickson.sage", "lie-algebras.sage")

We work with the orthonormal basis (with respect to the bi-invariant metric) $X_0, \ldots, X_{13}$ of $\mathfrak g_2$ which was also used in `G2-left-invariant-metric.ipynb`.

In [3]:
on_basis_biinv = load("on_basis.sobj")

We keep the notations
\begin{align*}
\mathfrak k_0 & = \mathbb R X_0 \oplus \mathbb R X_1 \oplus \mathbb R X_2 \\
\mathfrak m_0 & = \mathbb R X_3 \oplus \mathbb R X_4 \oplus \mathbb R X_5 \\
\mathfrak m_1 & = \mathbb R X_6 \oplus \mathbb R X_7 \oplus \mathbb R X_8 \oplus \mathbb R X_9 \\
\mathfrak m_2 & = \mathbb R X_{10} \oplus \mathbb R X_{11} \oplus \mathbb R X_{12} \oplus \mathbb R X_{13}
\end{align*}

Let $K_0 \simeq \operatorname{SU}(2)$ be the subgroup of $G_2$ with Lie algebra $\mathfrak k_0$ (which we have already proven to be a Lie subalgebra of $\mathfrak g_2$). If $\mathfrak m = \mathfrak m_0 \oplus \mathfrak m_1 \oplus \mathfrak m_2$, then 
$$\mathfrak g_2 = \mathfrak k_0 \oplus \mathfrak m$$
is a reductive decomposition.

Let us consider the isometric action of $G_2$ on $ZD(\mathbb S)$. Recall that this action is transitive, so $ZD(\mathbb S) = G_2 \cdot u_0 \simeq V_2(\mathbb R^7)$ for any $u_0 \in ZD(\mathbb S)$. Moreover, if $u_0 = e_1 + e_{10}$ then the isotropy group at $u_0$ is $K_0$. Indeed,

In [4]:
e = identity_matrix(16).columns()
u0, v0 = e[1] + e[10], e[4] - e[15]
# check u0 is a zero divisor
display(prod_S(u0, v0) == 0)
# check that k0 is the isotropy subalgebra
all(block_diagonal_matrix([X, X]) * u0 == 0 for X in on_basis_biinv[:3]) #

True

True

The metric in $ZD(\mathbb S)$ induces a $G_2$-invariant metric $g$ on $V_2(\mathbb R^7) = G_2 / \operatorname{SU}(2)$, which is determined by the inner product on $\mathfrak m$ given by
$$g(X_i, X_j) = (X_i \cdot u_0)^T (X_i \cdot u_0), \qquad i,j = 3, \ldots, 13.$$

In [5]:
m_initial_basis = on_basis_biinv[3:]
V2R7_metric_matrix = matrix(11, 11, lambda i, j: (block_diagonal_matrix([m_initial_basis[i],m_initial_basis[i]]) * u0
                            ) * (block_diagonal_matrix([m_initial_basis[j],m_initial_basis[j]]) * u0))
display(V2R7_metric_matrix.is_diagonal())
V2R7_metric_matrix.diagonal()

True

[1/3, 1/3, 2/3, 1/2, 1/2, 1/2, 1/2, 1/6, 1/6, 1/6, 1/6]

Thus, the $G_2$-invariant metric on $V_2(\mathbb R^7)$ is 
$$g = \frac13 (X^3 \otimes X^3 + X^4 \otimes X^4) + \frac23 X^5 \otimes X^5 + \frac12 \sum_{i=6}^9 X^i \otimes X^i + \frac16 \sum_{i=10}^{13} X^i \otimes X^i.$$

We will study the the family of $G_2$-invariant metrics $g_r$, $r >0$, given by
$$\frac13 (X^3 \otimes X^3 + X^4 \otimes X^4) + r X^5 \otimes X^5 + \frac12 \sum_{i=6}^9 X^i \otimes X^i + \frac16 \sum_{i=10}^{13} X^i \otimes X^i$$
Notice that $g = g_{\frac23}$.

## Definition of $g_r$

We define a function to compute $g_r$, which is called `metric_m`.

In [6]:
r = var("r")
diagonal = [1/3, 1/3, r, 1/2, 1/2, 1/2, 1/2, 1/6, 1/6, 1/6, 1/6]

def S(X):
    return sum([diagonal[i] * biinv_metric(X, m_initial_basis[i]) * m_initial_basis[i] for i in range(11)])

def metric_m(X, Y):
    return biinv_metric(X, S(Y))

In [7]:
metric_m_matrix = matrix(11, 11, lambda i, j: metric_m(m_initial_basis[i], m_initial_basis[j]))
display(metric_m_matrix.is_diagonal())
metric_m_matrix.diagonal()

True

[1/3, 1/3, r, 1/2, 1/2, 1/2, 1/2, 1/6, 1/6, 1/6, 1/6]

We check that $g_r$ is indeed an $\operatorname{Ad}(K_0)$-invariant inner product on $\mathfrak m$.

In [8]:
# define an orthonormal basis of m
m_on_basis = gram_schmidt(m_initial_basis, metric_m)

def ad_matrix_m(X):
    return matrix(11, 11, lambda i, j: metric_m(lie_bracket(X, m_on_basis[j]), m_on_basis[i]))

%time all(ad_matrix_m(X).is_skew_symmetric() for X in on_basis_biinv[:3])

CPU times: user 4.45 s, sys: 434 μs, total: 4.45 s
Wall time: 4.45 s


True

## The geometry of $(V_2(\mathbb R^7), g_r)$

Recall that the sectional curvature of $g_r$ can be computed as
\begin{align*}
g_r(R(X, Y) X, Y)  = & -\frac34 g_r( [X, Y]_{\mathfrak m}, [X, Y]_{\mathfrak m}) - \frac12 g_r [X, [X, Y]_{\mathfrak m} ]_{\mathfrak m}, Y) \\
& -\frac12 g_r( [Y, [Y, X]_{\mathfrak m} ]_{\mathfrak m}, X) + g_r( U(X, Y), U(X, Y)) \\
& - g_r( U(X, X), U(Y, Y) ) + g_r( Y, [ [X, Y]_{\mathfrak h}, X]_{\mathfrak m})
\end{align*}
for $X, Y \in \mathfrak m$, where $R$ is the Riemannian curvature tensor of $g_r$ and $U: \mathfrak m \times \mathfrak m \to \mathfrak m$ is implicitly given by the formula
$$2 g_r( U(X, Y), Z ) = g_r( [Z, X]_{\mathfrak m}, Y ) + g_r( X, [Z, Y]_{\mathfrak m} )$$
for $X, Y, Z \in \mathfrak m$. We can also compute the Ricci tensor $\operatorname{Ric}$ of $g_r$ as
\begin{align}
  \operatorname{Ric}(X, X) = & -\frac12 \sum_i \{\| [X, Y_i]_{\mathfrak m}\|^2 + g_r([X, [X, Y_i]_{\mathfrak m} ]_{\mathfrak m}, Y_i) + 2 g_r([ X, [X, Y_i]_{\mathfrak h} ]_{\mathfrak m}, Y_i)\} \\
   & + \frac14 \sum_{i, j} g_r([Y_i, Y_j]_{\mathfrak m}, X)^2 
  - g_r([Z, X]_{\mathfrak m}, X), 
\end{align}
for $X \in \mathfrak m$, where $\{Y_i\}$ is an orthonormal basis of $\mathfrak m$ and $Z = \sum_i U(Y_i, Y_i)$.

First of all we define the projections $[-, -]_{\mathfrak k_0}$ and $[-, -]_{\mathfrak m}$

In [9]:
def proj_k0(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in m_initial_basis])

def proj_m(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in on_basis_biinv[:3]])

Next we define $U$ via the function `U_tensor`.

In [10]:
def U_tensor(X, Y):
    return (1/2) * sum([(metric_m(proj_m(lie_bracket(Z,X)),Y) 
                         + metric_m(X, proj_m(lie_bracket(Z, Y)))) * Z for Z in m_on_basis])

In [11]:
# One can check that the metric is not naturally reductive
# long time
# %time all([U_tensor(X, Y) == 0 for X in m_on_basis for Y in m_on_basis])

Now we define de function `sec_curv` to compute $g_r(R_{X, Y}X, Y)$. Notice that when `X, Y` is an orthonormal $2$-frame, `sec_curv(X, Y)` computes the sectional curvature of the plane generated by `X, Y`.

In [12]:
def sec_curv(X, Y):
    sum1 = -(3/4) * metric_m(proj_m(lie_bracket(X, Y)), proj_m(lie_bracket(X, Y)))
    sum2 = -(1/2) * metric_m(proj_m(lie_bracket(X, proj_m(lie_bracket(X, Y)))), Y)
    sum3 = -(1/2) * metric_m(proj_m(lie_bracket(Y, proj_m(lie_bracket(Y, X)))), X)
    sum4 = metric_m(U_tensor(X, Y), U_tensor(X, Y))
    sum5 = -metric_m(U_tensor(X, X), U_tensor(Y, Y))
    sum6 = metric_m(Y, proj_m(lie_bracket(proj_k0(lie_bracket(X, Y)), X)))
    return sum1 + sum2 + sum3 + sum4 + sum5 + sum6

For example, we can see that if $r > \frac49$, then the sectional curvature of the plane $\mathbb R X_3 \oplus \mathbb R X_4$ is negative (this is used in the proof of Theorem 4.4).

In [13]:
%time sec_curv(m_on_basis[0], m_on_basis[1])

CPU times: user 1.68 s, sys: 33 μs, total: 1.68 s
Wall time: 1.68 s


-9/4*r + 1

Now we implement the following functions in order to compute the Ricci tensor.

In [14]:
sum_Uii = sum([U_tensor(X, X) for X in m_on_basis])
%time XiXj = {(i,j): proj_m(lie_bracket(m_on_basis[i], m_on_basis[j])).canonicalize_radical() for i in range(11) for j in range(11)}

CPU times: user 7.74 s, sys: 150 ms, total: 7.89 s
Wall time: 6.73 s


In [15]:
def ricci_quadratic_form(X):
    XXi_m = [proj_m(lie_bracket(X, Y)) for Y in m_on_basis]
    XXXi_mm = [proj_m(lie_bracket(X, XXi_m[i])) for i in range(11)]
    XXXi_hm = [proj_m(lie_bracket(X,proj_k0(lie_bracket(X, Y)))) for Y in m_on_basis]
    sum1 = -(1/2) * sum([metric_m(Y, Y) for Y in XXi_m])
    sum2 = -(1/2) * sum([metric_m(XXXi_mm[i], m_on_basis[i]) for i in range(11)])
    sum3 = -sum([metric_m(XXXi_hm[i], m_on_basis[i]) for i in range(11)])
    sum4 = (1/4) * sum([metric_m(XiXj[(i,j)], X)^2 for i in range(11) for j in range(11)])
    sum5 = -metric_m(proj_m(lie_bracket(sum_Uii, X)), X)
    return sum1 + sum2 + sum3 + sum4 + sum5

In [16]:
#def ricci_tensor(X, Y):
#    return (1/2) * (ricci_quadratic_form(X + Y) - ricci_quadratic_form(X) - ricci_quadratic_form(Y))

def ricci_tensor(X, Y):
    return (1/4) * (ricci_quadratic_form(X + Y) - ricci_quadratic_form(X - Y))

In [17]:
%time ricci_tensor(m_on_basis[7], m_on_basis[7])

CPU times: user 3.3 s, sys: 28 μs, total: 3.3 s
Wall time: 3.3 s


-3/2*r + 5

In [18]:
%time ricci_tensor(m_on_basis[2], m_on_basis[2])

CPU times: user 2.8 s, sys: 0 ns, total: 2.8 s
Wall time: 2.8 s


15/2*r

## Proof of Remark 3.4

Let $\pi_{ij} = \mathbb R X_i \oplus \mathbb R X_j$ for $3 \le i < j \le 13$. We compute the sectional curvature of $\pi_{ij}$ for the metric $g = g_{\frac23}$

In [19]:
# long time
sect_curv_dict_23 = {}
for i in range(3,14):
    for j in range(i+1, 14):
        sect_curv_dict_23[(i,j)] = sec_curv(m_on_basis[i-3], m_on_basis[j-3]).subs(r=2/3)
        print("sectional curvature of pi" + str(i) + str(j) + " added")
print("Done.")

sectional curvature of pi34 added
sectional curvature of pi35 added
sectional curvature of pi36 added
sectional curvature of pi37 added
sectional curvature of pi38 added
sectional curvature of pi39 added
sectional curvature of pi310 added
sectional curvature of pi311 added
sectional curvature of pi312 added
sectional curvature of pi313 added
sectional curvature of pi45 added
sectional curvature of pi46 added
sectional curvature of pi47 added
sectional curvature of pi48 added
sectional curvature of pi49 added
sectional curvature of pi410 added
sectional curvature of pi411 added
sectional curvature of pi412 added
sectional curvature of pi413 added
sectional curvature of pi56 added
sectional curvature of pi57 added
sectional curvature of pi58 added
sectional curvature of pi59 added
sectional curvature of pi510 added
sectional curvature of pi511 added
sectional curvature of pi512 added
sectional curvature of pi513 added
sectional curvature of pi67 added
sectional curvature of pi68 added
se

Finally, we check that the sectional curvature of $\pi_{i,j}$ is negative if and only if $(i, j) = (3, 4)$.

In [20]:
for key in sect_curv_dict_23.keys():
    if sect_curv_dict_23[key] < 0:
        print(key)

(3, 4)


## Proof of Theorem 4.2

Let us compute the Ricci tensor of $g_r$

In [21]:
Ric = matrix(SR, 11, 11)

# very long time
for i in range(11):
    for j in range(i, 11):
        Ric[i,j] = Ric[j, i] = ricci_tensor(m_on_basis[i], m_on_basis[j]).canonicalize_radical()
        print('entries ('+ str(i) + ',' +str(j) + ') and (' + str(j) + ',' + str(i) + ') complete')
print("Done.")

entries (0,0) and (0,0) complete
entries (0,1) and (1,0) complete
entries (0,2) and (2,0) complete
entries (0,3) and (3,0) complete
entries (0,4) and (4,0) complete
entries (0,5) and (5,0) complete
entries (0,6) and (6,0) complete
entries (0,7) and (7,0) complete
entries (0,8) and (8,0) complete
entries (0,9) and (9,0) complete
entries (0,10) and (10,0) complete
entries (1,1) and (1,1) complete
entries (1,2) and (2,1) complete
entries (1,3) and (3,1) complete
entries (1,4) and (4,1) complete
entries (1,5) and (5,1) complete
entries (1,6) and (6,1) complete
entries (1,7) and (7,1) complete
entries (1,8) and (8,1) complete
entries (1,9) and (9,1) complete
entries (1,10) and (10,1) complete
entries (2,2) and (2,2) complete
entries (2,3) and (3,2) complete
entries (2,4) and (4,2) complete
entries (2,5) and (5,2) complete
entries (2,6) and (6,2) complete
entries (2,7) and (7,2) complete
entries (2,8) and (8,2) complete
entries (2,9) and (9,2) complete
entries (2,10) and (10,2) complete
entr

In [22]:
display(Ric.is_diagonal())
Ric.diagonal()

True

[-3/2*r + 5,
 -3/2*r + 5,
 15/2*r,
 -3/2*r + 5,
 -3/2*r + 5,
 -3/2*r + 5,
 -3/2*r + 5,
 -3/2*r + 5,
 -3/2*r + 5,
 -3/2*r + 5,
 -3/2*r + 5]

It follows that $g_r$ is an Einstein metric if and only if $r = \frac59$.

In [23]:
solve(Ric[0,0] == Ric[2,2], r)

[r == (5/9)]

In [24]:
Ric.subs(r=5/9) == 25/6 * identity_matrix(11)

True

We also compute the scalar curvature and check that it is positive if and only of $r < \frac{20}{3}$.

In [25]:
scal = Ric.trace()
display(scal)
solve(scal, r)

-15/2*r + 50

[r == (20/3)]

## Proof of Theorem 4.4 - Part 0

Let $Y_3, \ldots, Y_{13}$ be the $g_r$-orthonormal basis of $\mathfrak m$ obtained from normalizing $X_3, \ldots, X_{13}$. We construct the polynomial 
\begin{equation*}
  F_r = g_r(R_{X, Y}X, Y) \in \mathbb R[x_0, \ldots, x_{21}],
\end{equation*}
where
\begin{align*}
  X = \sum_{i = 3}^{13} x_{i - 3} Y_i && Y = \sum_{i = 3}^{13} x_{i + 8} Y_i.
\end{align*} 



In [26]:
x_vars = [var("x" + str(i)) for i in range(22)]
X = sum(x_vars[i] * m_on_basis[i] for i in range(11)).canonicalize_radical()
Y = sum(x_vars[i+11] * m_on_basis[i] for i in range(11)).canonicalize_radical()

In [27]:
# long time
%time F = sec_curv(X, Y).canonicalize_radical()

CPU times: user 38.9 s, sys: 130 ms, total: 39 s
Wall time: 25.7 s


In [28]:
print(F)

1/2*(9*r - 4)*x0*x1*x11*x12 - 1/4*((9*r - 4)*x1^2 - 2*x10^2)*x11^2 - 1/4*((9*r - 4)*x0^2 - 2*x10^2)*x12^2 + 3/4*(r*x0^2 + r*x1^2 + r*x10^2)*x13^2 + 1/2*(x0^2 + x1^2 + x10^2)*x14^2 + 1/2*(x0^2 + x1^2)*x15^2 + 1/2*(x0^2 + x1^2)*x16^2 + 1/2*(x0^2 + x1^2 + x10^2)*x17^2 - 1/4*((9*r - 8)*x10^2 - 2*x0^2 - 2*x1^2)*x18^2 + 1/2*(x0^2 + x1^2 + x10^2)*x19^2 - 3/2*(r*x0*x11 + r*x1*x12)*x13*x2 + 3/4*(r*x11^2 + r*x12^2 + r*x14^2 + r*x15^2 + r*x16^2 + r*x17^2 + r*x18^2 + r*x19^2)*x2^2 + 1/4*(3*r*x2^2 + 2*x0^2 + 2*x1^2 + 2*x10^2)*x20^2 + 1/4*(3*r*x2^2 + 2*x0^2 + 2*x1^2)*x21^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x17^2 + 2*x11^2 + 2*x12^2 + 2*x15^2 + 2*x16^2 + 2*x18^2 + 2*x21^2)*x3^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x16^2 + 2*x11^2 + 2*x12^2 + 2*x14^2 + 2*x17^2 + 2*x19^2 + 2*x20^2)*x4^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x15^2 + 2*x11^2 + 2*x12^2 + 2*x14^2 + 2*x17^2 + 2*x19^2 + 2*x20^2)*x5^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x14^2 + 2*x11^2 + 2*x12^2 + 2*x15^2 + 2*x16^2 + 2*x18^2 + 2*x21^2)*x6^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x

In [29]:
#save(F, "F")

We check that $F$ is linear in $r$.

In [30]:
F.diff(r,2)

0

### Simplified expression for $F$

We get a better formula for $F$ using the `PolynomialRing()` function provided by SageMath. In particular, this allows us to order the monomials with the lexicographic order.

In [31]:
R.<x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21> = PolynomialRing(SR,order="lex")
r = var("r")
F1 = 1/2*(9*r - 4)*x0*x1*x11*x12 - 1/4*((9*r - 4)*x1^2 - 2*x10^2)*x11^2 - 1/4*((9*r - 4)*x0^2 - 2*x10^2)*x12^2 + 3/4*(r*x0^2 + r*x1^2 + r*x10^2)*x13^2 + 1/2*(x0^2 + x1^2 + x10^2)*x14^2 + 1/2*(x0^2 + x1^2)*x15^2 + 1/2*(x0^2 + x1^2)*x16^2 + 1/2*(x0^2 + x1^2 + x10^2)*x17^2 - 1/4*((9*r - 8)*x10^2 - 2*x0^2 - 2*x1^2)*x18^2 + 1/2*(x0^2 + x1^2 + x10^2)*x19^2 - 3/2*(r*x0*x11 + r*x1*x12)*x13*x2 + 3/4*(r*x11^2 + r*x12^2 + r*x14^2 + r*x15^2 + r*x16^2 + r*x17^2 + r*x18^2 + r*x19^2)*x2^2 + 1/4*(3*r*x2^2 + 2*x0^2 + 2*x1^2 + 2*x10^2)*x20^2 + 1/4*(3*r*x2^2 + 2*x0^2 + 2*x1^2)*x21^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x17^2 + 2*x11^2 + 2*x12^2 + 2*x15^2 + 2*x16^2 + 2*x18^2 + 2*x21^2)*x3^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x16^2 + 2*x11^2 + 2*x12^2 + 2*x14^2 + 2*x17^2 + 2*x19^2 + 2*x20^2)*x4^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x15^2 + 2*x11^2 + 2*x12^2 + 2*x14^2 + 2*x17^2 + 2*x19^2 + 2*x20^2)*x5^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x14^2 + 2*x11^2 + 2*x12^2 + 2*x15^2 + 2*x16^2 + 2*x18^2 + 2*x21^2)*x6^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x21^2 + 2*x11^2 + 2*x12^2 + 2*x14^2 + 2*x17^2 + 2*x19^2 + 2*x20^2)*x7^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x20^2 + 2*x11^2 + 2*x12^2 + 2*x15^2 + 2*x16^2 + 2*x18^2 + 2*x21^2)*x8^2 + 1/4*(3*r*x13^2 - (9*r - 8)*x19^2 + 2*x11^2 + 2*x12^2 + 2*x15^2 + 2*x16^2 + 2*x18^2 + 2*x21^2)*x9^2 - (x1*x10*x11 + x0*x10*x12)*x15 + (x0*x10*x11 - x1*x10*x12)*x16 + 1/2*(3*(3*r - 2)*x1*x10*x11 - 3*(3*r - 2)*x0*x10*x12 - 4*x0*x1*x16 - 2*(x0^2 - x1^2)*x15)*x18 - (2*x0*x1*x17 - (x0^2 - x1^2)*x14)*x19 + (2*x0*x1*x14 + (x0^2 - x1^2)*x17)*x20 - 1/2*(3*r*x10*x13*x2 + 2*x0*x10*x11 + 2*x1*x10*x12 - 4*x0*x1*x15 + 2*(x0^2 - x1^2)*x16)*x21 + 1/2*(3*(3*r - 2)*x10*x17*x18 - 3*r*x13*x14*x2 - 2*x10*x14*x21 - 2*(x0*x11 + x1*x12)*x14 - 3*((3*r - 2)*x1*x11 - (3*r - 2)*x0*x12)*x17 - 2*(x0*x11 - x1*x12 - x10*x16)*x19 - 2*(x1*x11 + x0*x12 + x10*x15)*x20)*x3 - 1/2*(3*r*x13*x15*x2 - 4*x10*x11*x12 + 4*x10*x17*x19 - 4*x10*x14*x20 + 2*(x0*x11 + x1*x12)*x15 - 3*((3*r - 2)*x1*x11 - (3*r - 2)*x0*x12)*x16 + (9*r*x10*x16 - 2*x0*x11 + 2*x1*x12)*x18 + 2*(x1*x11 + x0*x12)*x21 - (3*(3*r - 2)*x16*x17 - 2*x14*x15 + 2*x18*x19 - 2*x20*x21)*x3)*x4 - 1/2*(3*r*x13*x16*x2 - (9*r - 8)*x15*x16*x4 + 2*x10*x11^2 - 2*x10*x12^2 + 4*x10*x14*x19 + 4*x10*x17*x20 + 3*((3*r - 2)*x1*x11 - (3*r - 2)*x0*x12)*x15 + 2*(x0*x11 + x1*x12)*x16 - (9*r*x10*x15 + 2*x1*x11 + 2*x0*x12)*x18 - 2*(x0*x11 - x1*x12)*x21 + (3*(3*r - 2)*x15*x17 + 2*x14*x16 - 2*x18*x20 - 2*x19*x21)*x3)*x5 - 1/2*(3*(3*r - 2)*x10*x14*x18 + 3*r*x13*x17*x2 - (9*r - 8)*x14*x17*x3 + 2*x10*x17*x21 - 3*((3*r - 2)*x1*x11 - (3*r - 2)*x0*x12)*x14 + 2*(x0*x11 + x1*x12)*x17 - 2*(x1*x11 + x0*x12 + x10*x15)*x19 + 2*(x0*x11 - x1*x12 - x10*x16)*x20 + (3*(3*r - 2)*x14*x16 + 2*x15*x17 - 2*x18*x20 - 2*x19*x21)*x4 - (3*(3*r - 2)*x14*x15 - 2*x16*x17 - 2*x18*x19 + 2*x20*x21)*x5)*x6 - 1/2*(3*r*x13*x18*x2 - 2*(x0*x11 - x1*x12)*x15 - 2*(x1*x11 + x0*x12)*x16 + 2*(x0*x11 + x1*x12)*x18 + (3*(3*r - 2)*x1*x11 - 3*(3*r - 2)*x0*x12 - (9*r - 8)*x10*x18)*x21 + (3*(3*r - 2)*x17*x21 + 2*x14*x18 - 2*x15*x19 - 2*x16*x20)*x3 - (9*r*x16*x21 - 2*x11^2 + 2*x12^2 - 4*x14*x19 - 4*x17*x20)*x4 + (9*r*x15*x21 + 4*x11*x12 - 4*x17*x19 + 4*x14*x20)*x5 - (3*(3*r - 2)*x14*x21 - 2*x17*x18 - 2*x16*x19 + 2*x15*x20)*x6)*x7 - 1/2*(3*r*x13*x19*x2 - 2*x10*x14*x16 + 2*x10*x19*x21 + 2*(x0*x11 - x1*x12)*x14 - 2*(x1*x11 + x0*x12 + x10*x15)*x17 + 2*(x0*x11 + x1*x12)*x19 - 3*((3*r - 2)*x1*x11 - (3*r - 2)*x0*x12 - (3*r - 2)*x10*x18)*x20 - (9*r*x17*x20 + 2*x11^2 - 2*x12^2 - 4*x15*x18 - 4*x16*x21)*x3 + (3*(3*r - 2)*x16*x20 - 2*x14*x18 + 2*x15*x19 - 2*x17*x21)*x4 - (3*(3*r - 2)*x15*x20 - 2*x17*x18 - 2*x16*x19 + 2*x14*x21)*x5 + (9*r*x14*x20 + 4*x11*x12 - 4*x16*x18 + 4*x15*x21)*x6 - (3*(3*r - 2)*x20*x21 + 2*x14*x15 - 2*x16*x17 - 2*x18*x19)*x7)*x8 + 1/2*((9*r - 8)*x19*x20*x8 - 2*x10*x14*x15 - 2*x10*x20*x21 - 2*(x1*x11 + x0*x12)*x14 - 2*(x0*x11 - x1*x12 - x10*x16)*x17 - 3*((3*r - 2)*x1*x11 - (3*r - 2)*x0*x12 - (3*r - 2)*x10*x18)*x19 - (3*r*x13*x2 + 2*x0*x11 + 2*x1*x12)*x20 - (9*r*x17*x19 - 4*x11*x12 + 4*x16*x18 - 4*x15*x21)*x3 + (3*(3*r - 2)*x16*x19 + 2*x17*x18 - 2*x15*x20 - 2*x14*x21)*x4 - (3*(3*r - 2)*x15*x19 - 2*x14*x18 + 2*x16*x20 - 2*x17*x21)*x5 + (9*r*x14*x19 + 2*x11^2 - 2*x12^2 - 4*x15*x18 - 4*x16*x21)*x6 - (3*(3*r - 2)*x19*x21 - 2*x14*x16 - 2*x15*x17 + 2*x18*x20)*x7)*x9

In [32]:
print(F1)

(-9/4*r + 1)*x0^2*x12^2 + 3/4*r*x0^2*x13^2 + 1/2*x0^2*x14^2 + x0^2*x14*x19 + 1/2*x0^2*x15^2 - x0^2*x15*x18 + 1/2*x0^2*x16^2 - x0^2*x16*x21 + 1/2*x0^2*x17^2 + x0^2*x17*x20 + 1/2*x0^2*x18^2 + 1/2*x0^2*x19^2 + 1/2*x0^2*x20^2 + 1/2*x0^2*x21^2 + (9/2*r - 2)*x0*x1*x11*x12 + 2*x0*x1*x14*x20 + 2*x0*x1*x15*x21 + (-2)*x0*x1*x16*x18 + (-2)*x0*x1*x17*x19 + (-3/2*r)*x0*x2*x11*x13 - x0*x3*x11*x14 - x0*x3*x11*x19 + (9/2*r - 3)*x0*x3*x12*x17 - x0*x3*x12*x20 - x0*x4*x11*x15 + x0*x4*x11*x18 + (-9/2*r + 3)*x0*x4*x12*x16 - x0*x4*x12*x21 - x0*x5*x11*x16 + x0*x5*x11*x21 + (9/2*r - 3)*x0*x5*x12*x15 + x0*x5*x12*x18 - x0*x6*x11*x17 - x0*x6*x11*x20 + (-9/2*r + 3)*x0*x6*x12*x14 + x0*x6*x12*x19 + x0*x7*x11*x15 - x0*x7*x11*x18 + x0*x7*x12*x16 + (9/2*r - 3)*x0*x7*x12*x21 - x0*x8*x11*x14 - x0*x8*x11*x19 + x0*x8*x12*x17 + (-9/2*r + 3)*x0*x8*x12*x20 - x0*x9*x11*x17 - x0*x9*x11*x20 - x0*x9*x12*x14 + (9/2*r - 3)*x0*x9*x12*x19 + x0*x10*x11*x16 - x0*x10*x11*x21 - x0*x10*x12*x15 + (-9/2*r + 3)*x0*x10*x12*x18 + (-9/4*r + 1)